In [38]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import math
import collections
import wfdb

In [39]:
#Use records as dicts signal is a name such as 'ABP SYS'


def getSignal(signal,record):
    ind = record['sig_name'].index(signal)
    units = record['units'][ind]
    return record['p_signal'][:,ind],units

        
        



In [40]:
def detectHRArtifact(signal,units):
    """
    Parameters: < 20
                > 220
    """
    assert units.lower() == 'bpm'
    indices = []
    
    for index, signalValue in enumerate(signal):
        
        if signalValue < 20 or signalValue > 220:
            indices.append(index)
            
    return indices

    
def detectCVPArtifact(signal,units):
    """
    Parameters:  <  0
                 > 30
    """
    #assert units.lower() == 'bpm'
    indices = []
    
    for index, signalValue in enumerate(signal):
        
        if signalValue < 0 or signalValue > 30:
            indices.append(index)
            
    return indices


def detectRESPArtifact(signal,units):
    """
    Parameters: 
                 > 50
    """
    #assert units.lower() == 'bpm'
    indices = []
    
    for index, signalValue in enumerate(signal):
        
        if signalValue > 50:
            indices.append(index)
            
    return indices
    
    
def detectSpO2Artifact(signal,units):
    """
    Parameters: 
                 < 50
    """
    #assert units.lower() == 'bpm'
    indices = []
    for index, signalValue in enumerate(signal):
        
        if signalValue < 50:
            indices.append(index)
            
    return indices


    


SyntaxError: invalid syntax (<ipython-input-41-075a68a6b6bb>, line 1)

In [42]:
def detectNBPArtifact(sys,dias,mean,sysunits,diasunits,meanunits,fs):
    sysIndices = []
    diasIndices = []
    meanIndices = []
    for index, dias in enumerate(dias):
        if dias > 250:
            diasIndices.append(index)
        if mean[index] > 250:
            meanIndices.append(index)
            
    return [],diasIndices,meanIndices

In [43]:
def detectABPArtifact(sys,dias,mean,sysunits,diasunits,meanunits,fs):
    """
        
    NEED TO DETERMINE HOW TO DOWNSAMPLE 1HZ SIGNAL
    Using a similar approach then Sun et al. 
    
    Condition 1: SBP >= 300 or SBP <= 20
    Condition 2: SBP - DBP <= 5
                 SBP - DBP > 200
    
    Condition 3: DBP <=5 or DBP >= 225
    
    Condition 4: BP beat-to beat > 20mmhg for Ps,Pd, mean
    
    Condition 5: += 80 mmhg / min change
    
    Note: not complete for 1Hz condition 5, need to determine how to downsample to 1/60Hz
    """

    assert (sysunits.lower() == diasunits.lower() == meanunits.lower() == 'mmhg')
    
    sysIndices =[]
    diasIndices = []
    meanIndices = []
    
    for index,sysSignalValue in enumerate(sys):
        # Condition 1
        if sysSignalValue >= 300 or sysSignalValue <= 20:
            sysIndices.append(index)
        # Condition 2    
        if signalValue - dias[index] <= 5 or signalValue - dias[index] > 200:
            sysIndices.append(index)
            diasIndices.append(index)
            meanIndices.append(index)
    # Condition 3        
    for index, diasSignalValue in enumerate(dias):
        if diasSignalValue <=5 or diasSignalValue >= 225:
            diasIndices.append(index)
            
            
            
            
            
    def detectDelta(signal,delta):
        flaggedsig = []
        for index, sigVal in enumerate(signal[:-1]):
            if not np.isnan(sigVal) and not np.isnan(signal[index+1]):
                if abs(signal[index+1] - sigVal) > delta:
                    #flaggedsig.append(index)
                    flaggedsig.append(index+1)
        return flaggedsig
    
    
    if round(fs,4) == .0167:
        for index, signal in enumerate(sys[:-1]):
            # Sys
            if not np.isnan(signal) and not np.isnan(signal[index+1]):
                if abs(sys[index+1]-signal) > 80:
                    sysIndices.append(index+1)
            # Dias        
            if not np.isnan(dias[index]) and not np.isnan(dias[index+1]):
                if abs(dias[index+1]-dias[index]) > 80:
                    diasIndices.append(index+1)
            # Mean        
            if not np.isnan(mean[index]) and not np.isnan(mean[index+1]):
                if abs(mean[index+1]-mean[index]) > 80:        
                    meanIndices.append(index+1)
                    
    
    
    return sysIndices,diasIndices,meanIndices
        
    

In [44]:
# REPLACE NANS

3
            
        
            


def interpolateNans(signal,sf):
    
    start = startOfSignal(signal)
    end = len(signal) - 1
    
    if round(sf,4) == .0167:
        
        temp = 0
            
        for index in range(start,end):
            
            gap = 0
            gaplist = []
                    
            if not np.nan(signal[index]):
                
                if gap != 0:
                    
                
                    lastVal = signal[index]
                
            
            
            
        
        

    


In [45]:

def interpolateNans(signal):
    def nan_helper(y):

        return np.isnan(y), lambda z: z.nonzero()[0]


    nans, x= nan_helper(signal)
    signal[nans]= np.interp(x(nans), x(~nans), signal[~nans])
    
    
    return signal


In [55]:

def getSignal(signal,record):
    ind = record['sig_name'].index(signal)
    units = record['units'][ind]
    return record['p_signal'][:,ind],units


file = '/Users/Arcrank/Documents/Work/mimic3wdb/matched_numeric/p03/p032226/p032226-2201-05-17-17-46n'
re = wfdb.rdrecord(file)
d = re.__dict__

HR,units = getSignal('HR',d)


# HR[0] = np.nan
# copy = HR.copy()


# fixed = interpolateNans(copy2)

# for index,s in enumerate(copy2):
#     print(s,copy[index])

In [68]:
def detectGaps(signal,sf):
    if round(sf,4) == .0167:
        #operate only from start to end
        nanVals = np.isnan(signal)
        cache = 0
        maxcache = 0
        for index,value in enumerate(signal):
            
            if cache > maxcache:
                maxcache = cache
            if cache > 60:
                print(index-1,'gaps too large')
                
            if np.isnan(value):
                cache +=1
                
            else:
                cache = 0
                
        return maxcache
            
for signal in HR:
    print(np.isnan(signal))

False
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

...

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

15


60
